<br>
<font size='6'><b>Python with Arduino</b></font><br><br>

<table style="border-style: hidden; border-collapse: collapse;" width = "80%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 50% style="border-style: hidden; border-collapse: collapse;">

        </td>
        <td width = 30%>
        by Seungchul Lee<br>
        iSystems Design Lab.<br>http://isystems.unist.ac.kr/<br>UNIST
        </td>
    </tr>
</table>

Table of Contents
<div id="toc"></div>

# 0. Installation
<br>
__ANACONDA Install Confirmation__
<br>

Before getting started with serial communication with Python, you must make sure pip is installed. However, pip will be automatically installed if ANACONDA is installed. Thus, you will have to do this step _only if you have not done it last time_. Below is the link for the instructions given last time.


http://nbviewer.jupyter.org/github/i-systems/IoT/blob/master/Mechatronics%20Class/HW/Python%20Installation%20Instructions.ipynb

<br>

__pip pyserial Installation__


<img src = "./image_files/command screen.png" width = 350>

Open the cmd window (press the windows key + r and type 'cmd'), and type in the following:
<br>
```c
python -m pip install pyserial
```

<img src = "./image_files/cmd screen V2.png" width = 550>


The download process will begin soon, and the pyserial module will be successively installed.

When you are finished installing pyserial, install bs4 using the same method. Type the following into the command window

```c
python -m pip install bs4
```



# 1. LED with python


<br>
Starting from this section, we will be using Python along with Arduino. As mentioned in previous classes, the codes being uploaded to the Arduino may have some limitations regarding the functionality. Starting from simple tasks such as byte sending through serial communication, we will be able to plot real time graphs based on the streamed data from Arduino.

<br>
$$
\large \text{Python in PC} \quad \overset{\text{Serial}}{\longleftrightarrow} \quad \text{Arduino} \quad \overset{\text{Digital Output}}{\longleftrightarrow} \quad \text{LED}$$

<br>

<img src = "./image_files/Python_wArduino_LED_fritz.png" width = 350>

<font size='4'><b>Lab 1: LED ON/OFF - Transmit (numeric) data from Python</b></font>

__Arduino code__

```c
int ON = 0;
const int pin = 9;

void setup() {
  pinMode(pin, OUTPUT);
  Serial.begin(9600);
}

void loop() {    
  if (Serial.available() > 0) {
    ON = Serial.read();
    
    if (ON == 1)
      digitalWrite(pin, HIGH);
    else if (ON == 0)
      digitalWrite(pin, LOW);

    Serial.println(ON);
  }
}
```

__Python code__

Before running this section, please make sure you are using the correct port. The sample code will be using 'COM4'.

In [ ]:
import serial
import time

ser = serial.Serial('COM4', 9600, timeout=1)

In [ ]:
ser.write(bytes([1]))

In [ ]:
ser.write(bytes([0]))

You have to close ser to disconnect. Otherwise other devices cannot use this serial communication. 

In [ ]:
ser.close()

Now we are ready to combine all into python with serial communication to Arduino to control LEDs.

In [ ]:
ser = serial.Serial('COM4', 9600, timeout=1)

for i in range(10):
    ser.write(bytes([1]))
    time.sleep(0.5)
    ser.write(bytes([0]))
    time.sleep(0.5)

ser.close()    

<font size='4'><b>Lab 2: LED ON/OFF - Transmit (string) data from Python</b></font>

__Arduino code__

```c
String LEDcmd = "";
const int pin = 9;

void setup() {
  pinMode(pin, OUTPUT);
  Serial.begin(9600);
}

void loop() {
  if (Serial.available() > 0) {
    LEDcmd = Serial.readStringUntil('\n');

    if (LEDcmd == "ON")
      digitalWrite(pin, HIGH);
    else if (LEDcmd == "OFF")
      digitalWrite(pin, LOW);
  }
}
```

__Python code__

In [ ]:
str('ON').encode('UTF-8')

In [ ]:
ser = serial.Serial('COM4', 9600, timeout=1)

In [ ]:
ser.write(str('ON').encode('UTF-8'))

In [ ]:
ser.write(str('OFF').encode('UTF-8'))

In [ ]:
ser.close()

In [ ]:
ser = serial.Serial('COM14', 9600, timeout=1)

for i in range(10):
    ser.write(str('ON').encode('UTF-8'))
    time.sleep(0.5)
    ser.write(str('OFF').encode('UTF-8'))
    time.sleep(0.5)

ser.close()

<font size='4'><b>Lab 3: LED Brightness</b></font>

__Arduino code__

```c
int brightness = 0;
const int pin = 9;

void setup() {
  pinMode(pin, OUTPUT);
  Serial.begin(9600);  
}

void loop() {
  if (Serial.available() > 0) {
    brightness = Serial.read();
    analogWrite(pin, brightness);
  }
}
```
<br>
__Python code__

In [ ]:
ser = serial.Serial('COM18', 9600, timeout=1)

for i in range(255):
    ser.write(bytes([i]))
    time.sleep(0.1)

for i in range(255):
    ser.write(bytes([255-i])) 
    time.sleep(0.1)    
    
ser.close() 

<font size='4'><b>Lab 4: Current Time on LCD Display</b></font>

<img src = "./image_files/lcd_new_scheme.png" width = 500>

__Arduino code__

```c
#include <LiquidCrystal.h>

LiquidCrystal lcd(12, 11, 5, 4, 3, 2);
String LCDRead = "";

void setup() {
  // set up the LCD's number of columns and rows:
  lcd.begin(16, 2);
  lcd.setCursor(0,0);
  // Print a message to the LCD.
  lcd.print("The current time");
  Serial.begin(9600);
}

void loop() {
  if (Serial.available()) {

    LCDRead = Serial.readStringUntil('\n');

    lcd.setCursor(0,1);
    lcd.print(LCDRead);    
  }
}
```

__Python code__

- Displaying current time once

In [ ]:
import serial
import time
from datetime import datetime

ser = serial.Serial('COM18', 9600, timeout=1)

In [ ]:
print(datetime.now().strftime('%H:%M:%S'))
ser.write((datetime.now().strftime('%H:%M:%S') + '\n').encode('UTF-8'))

In [ ]:
ser.close()

- Displaying the time for 15 seconds

In [ ]:
import serial
import time
from datetime import datetime

ser = serial.Serial('COM18', 9600, timeout=1)

for i in range(15):
    print(datetime.now().strftime('%H:%M:%S').encode('UTF-8'))
    ser.write((datetime.now().strftime('%H:%M:%S') + '\n').encode('UTF-8'))
    time.sleep(1)
    
ser.close()

<font size='4'><b>Lab 5: Current temperature on LCD Display from the internet</b></font>

__Arduino code__

```c
#include <LiquidCrystal.h>

LiquidCrystal lcd(12, 11, 5, 4, 3, 2);
String LEDRead = "";

void setup() {  
  lcd.begin(16,2);  
  Serial.begin(9600);
}

void loop() {
  if (Serial.available()) {

    LEDRead = Serial.readStringUntil('\n');
    lcd.clear();

    lcd.setCursor(0,0);
    lcd.print("Temperature in");
    lcd.setCursor(0,1);
    lcd.print("Ulsan: ");
    lcd.setCursor(7,1);
    lcd.print(LEDRead);
    lcd.setCursor(9,1);
    lcd.print("Degrees");
  }
}
```

<br>
__Python code__

In [ ]:
import serial
import time

ser = serial.Serial('COM14', 9600, timeout=1)

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

url = 'https://www.google.co.kr/search?q=ulsan+temperature'

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
html = urlopen(req).read()

soup = BeautifulSoup(html, 'html.parser')
result = soup.find_all('span', 'wob_t')

for i in range(len(result)):
    print(result[i].text)
    
ser.write(str(result[0].text[:-2]).encode('utf-8'))

# 2. Servo Motor with python

<br>
$$
\large \text{Python in PC} \quad \overset{\text{serial}}{\longleftrightarrow} \quad \text{arduino} \quad \overset{\text{PWM}}{\longleftrightarrow} \quad \text{servo}$$

<br>
<img src = "./image_files/servo V2.jpg" width = 400>

__Arduino code__

```c
#include <Servo.h>

Servo myservo;
const int pint = 9;
int pos = 0;

void setup() {
  myservo.attach(pin);
  myservo.write(0);
  Serial.begin(9600);
}

void loop() {
  if (Serial.available() > 0) {
    pos = Serial.read();
    myservo.write(pos);
    delay(50);
  }
}
```

__Python code__

In [ ]:
ser = serial.Serial('COM18', 9600, timeout=1)

In [ ]:
ser.write(bytes([90]))

In [ ]:
ser.write(bytes([180]))

In [ ]:
ser.close()

In [ ]:
ser = serial.Serial('COM18', 9600, timeout=1)

for angle in range(0,180):
    ser.write(bytes([angle]))
    time.sleep(0.1)
    
for angle in range(180,0,-1):
    ser.write(bytes([angle]))
    time.sleep(0.1)
    
ser.close()

In [ ]:
import serial
import time

def move(angle):
    if (0 <= angle <= 180):
        ser.write(bytes([angle]))
    else:
        print("Servo angle must be an integer between 30 and 180.\n")

# Start the serial port to communicate with arduino
ser = serial.Serial('COM18', 9600, timeout=1)

print("The initial servo angle is 30, type 'end' if you want to stop")

while 1:
    angle = input("Enter your angle: ")
    if angle == "end":
        print("Finished")
        ser.close()
        break
    else:
        move(int(angle))
        time.sleep(0.01)
        print("The current servo angle is " + angle)


# 3. Data Streaming with Python

## 3.1 MPU6050 and layout

Reference: http://playground.arduino.cc/Main/MPU-6050

The InvenSense MPU-6050 sensor contains a MEMS accelerometer and a MEMS gyro in a single chip. It is very accurate, as it contains 16-bits analog to digital conversion hardware for each channel. Therefor it captures the x, y, and z channel at the same time. The sensor uses the I2C-bus to interface with the Arduino.

<table style="border-style: hidden; border-collapse: collapse;" width = "90%"> 
    <tr style="border-style: hidden; border-collapse: collapse;">
        <td width = 25% style="border-style: hidden; border-collapse: collapse;">
<img src = "./image_files/mpu6050.jpg" width = 150>
        </td>
        <td width = 65%>
<img src = "./image_files/circuit.png" width = 500>
        </td>
    </tr>
</table>





## 3.2. Plotting Saved Data in buffer from Arduino

Before plotting the streaming data from the MPU6050, we will try plotting data after saving it in a buffer. The example code below saves 50 data sets before actually plotting it.

<br>
__Arduino Code__
```c
#include <Wire.h>

const int MPU = 0x68;   //Default address of I2C for MPU 6050
int16_t AcX, AcY, AcZ;

void setup() {
  Wire.begin();                   // Wire library initialization
  Wire.beginTransmission(MPU);    // Begin transmission to MPU
  Wire.write(0x6B);               // PWR_MGMT_1 register
  Wire.write(0);                  // MPU-6050 to start mode
  Wire.endTransmission(true);
  Serial.begin(9600);
}

void loop() {
  Wire.beginTransmission(MPU);      // Start transfer
  Wire.write(0x3B);                 // register 0x3B (ACCEL_XOUT_H), records data in queue
  Wire.endTransmission(false);      // Maintain connection
  Wire.requestFrom(MPU, 14, true);  // Request data to MPU
  //Reads byte by byte
  AcX = Wire.read() << 8 | Wire.read(); // 0x3B (ACCEL_XOUT_H) & 0x3C (ACCEL_XOUT_L)
  AcY = Wire.read() << 8 | Wire.read(); // 0x3D (ACCEL_YOUT_H) & 0x3E (ACCEL_YOUT_L)
  AcZ = Wire.read() << 8 | Wire.read(); // 0x3F (ACCEL_ZOUT_H) & 0x40 (ACCEL_ZOUT_L)

  //Prints values on Serial
  Serial.print(AcX);
  Serial.print(","); 
  Serial.print(AcY);
  Serial.print(","); 
  Serial.println(AcZ);
  delay(20);
}
```

<br>
__Python Code__

`> python -m pip install drawnow`

In [3]:
import serial          
import numpy as np     
import matplotlib.pyplot as plt 
%matplotlib qt 
from drawnow import *

ser = serial.Serial('COM5',9600)     

# Define Variables
Accx = []
Accy = []
Accz = []

plt.ion()   # Tell matplotlib you want interactive mode to plot live data

In [4]:
def makeFig():      # Create a function that makes our desired plot
    # plt.ylim(80,90)
    plt.figure(1)
    plt.subplot(311)                    # Set y min and max values
    plt.title('Saved data')             # Plot the title
    plt.grid(True)                      # Turn the grid on
    plt.ylabel('Acceleration')          # Set ylabels
    plt.plot(Accx, 'bo-', label='X')    # plot the temperature
    plt.legend(loc='upper left')        # plot the legend
    
    plt.subplot(312)                    # Set y min and max values
    plt.grid(True)                      # Turn the grid on
    plt.ylabel('Acceleration')          # Set ylabels
    plt.plot(Accy, 'ro-', label='Y')    # plot the temperature
    plt.legend(loc='upper left')          
    
    plt.subplot(313)                    # Set y min and max values
    plt.grid(True)                      # Turn the grid on
    plt.ylabel('Acceleration')          # Set ylabels
    plt.plot(Accz, 'go-', label='Z')    # plot the temperature
    plt.legend(loc='upper left')          

    plt.show()

In [5]:
for i in range (51):               # Wait until the buffer is filled up to 50 values
    while (ser.inWaiting() == 0):   
        pass                        
        
    arduinoString = ser.readline().decode("utf-8") #.strip()
    
    dataArray = (arduinoString.split(','))      # Since the values are being sent consecutively, 
                                                # the values must be split into x,y,and z
    # Data conversion from string to float
    temp1 = float(dataArray[0])             
    temp2 = float(dataArray[1]) 
    temp3 = float(dataArray[2])
    
    # Convert second element to floating number and put in P
    Accx.append(temp1)                      
    Accy.append(temp2)                      
    Accz.append(temp3)
                      
    plt.pause(.001)  # Pause Briefly. Important to keep drawnow from crashing    

In [ ]:
drawnow(makeFig)        # Call drawnow to draw the graph with the data from the buffer

In [7]:
ser.close()

<img src = "./image_files/live_2.png" style="border:1px solid black" width = 600>

## 3.3. Plotting Steaming Data from Arduino

__Arduino code__

```c
#include <Wire.h>

const int MPU = 0x68;   //Default address of I2C for MPU 6050
int16_t AcX, AcY, AcZ;

void setup() {
  Wire.begin();                   // Wire library initialization
  Wire.beginTransmission(MPU);    // Begin transmission to MPU
  Wire.write(0x6B);               // PWR_MGMT_1 register
  Wire.write(0);                  // MPU-6050 to start mode
  Wire.endTransmission(true);
  Serial.begin(9600);
}

void loop() {
  Wire.beginTransmission(MPU);      // Start transfer
  Wire.write(0x3B);                 // register 0x3B (ACCEL_XOUT_H), records data in queue
  Wire.endTransmission(false);      // Maintain connection
  Wire.requestFrom(MPU, 14, true);  // Request data to MPU
  //Reads byte by byte
  AcX = Wire.read() << 8 | Wire.read(); // 0x3B (ACCEL_XOUT_H) & 0x3C (ACCEL_XOUT_L)
  AcY = Wire.read() << 8 | Wire.read(); // 0x3D (ACCEL_YOUT_H) & 0x3E (ACCEL_YOUT_L)
  AcZ = Wire.read() << 8 | Wire.read(); // 0x3F (ACCEL_ZOUT_H) & 0x40 (ACCEL_ZOUT_L)

  //Prints values on Serial
  Serial.print(AcX);
  Serial.print(","); 
  Serial.print(AcY);
  Serial.print(","); 
  Serial.println(AcZ);
  delay(20);
}
```

__Python code__

In [1]:
# Plot 3 signals

import serial
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

ser = serial.Serial('COM5', 9600)

len = 200
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(xlim=(0,len), ylim=(-2, 2))

plt.title('Real-time sensor data')
plt.xlabel('Data points')
plt.ylabel('Acceleration [G]')
ax.grid(True)

graphX, = ax.plot([], [], 'b', label = 'X')
ax.legend(loc='upper right')
graphY, = ax.plot([], [], 'r', label = 'Y')
ax.legend(loc='upper right')
graphZ, = ax.plot([], [], 'g', label = 'Z')
ax.legend(loc='upper right')

time = list(range(0, len + 1))
accX = []
accY = []
accZ = []

for i in range(0, len + 1):
    accX.append(0)
    accY.append(0)
    accZ.append(0)

def init():
    graphX.set_data([], [])
    graphY.set_data([], [])
    graphZ.set_data([], [])
    return graphX, graphY, graphZ

def animate(i):
    global time, accX, accY, accZ

    while (ser.inWaiting() == 0):
        pass

    arduinoString = ser.readline().decode("utf-8")
    dataArray = (arduinoString.split(','))

    accX.append(float(dataArray[0])/(32767/2))
    accX.pop(0)
    accY.append(float(dataArray[1])/(32767/2))
    accY.pop(0)
    accZ.append(float(dataArray[2])/(32767/2))
    accZ.pop(0)

    graphX.set_data(time, accX)
    graphY.set_data(time, accY)
    graphZ.set_data(time, accZ)

    return graphX, graphY, graphZ

delay = 20
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               interval=delay, blit=True)

plt.show()     

In [2]:
ser.close()

<img src = "./image_files/MPU_plot.png" style="border:1px solid black" width = 600>

In [ ]:
# Subplot

import serial
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

ser = serial.Serial('COM5', 9600)

len = 200
fig = plt.figure(figsize=(12, 9))

ax1 = fig.add_subplot(3, 1, 1)
ax2 = fig.add_subplot(3, 1, 2)
ax3 = fig.add_subplot(3, 1, 3)

graphX, = ax1.plot([], [], 'b', label = 'X')
graphY, = ax2.plot([], [], 'r', label = 'Y')
graphZ, = ax3.plot([], [], 'g', label = 'Z')
axes = [ax1, ax2, ax3]

for ax in axes:
    ax.set_xlim(0, len)
    ax.set_ylim(-2, 2)
    ax.set_ylabel('Acceleration [G]')
    ax.legend(loc='upper right')
    ax.grid(True)

ax1.set_title('Real-time sensor data')
ax3.set_xlabel('Data points')
    
time = list(range(0, len + 1))
accX = []
accY = []
accZ = []

for i in range(0, len + 1):
    accX.append(0)
    accY.append(0)
    accZ.append(0)

def init():
    graphX.set_data([], [])
    graphY.set_data([], [])
    graphZ.set_data([], [])
    return graphX, graphY, graphZ

def animate(i):
    global time, accX, accY, accZ

    while (ser.inWaiting() == 0):
        pass

    arduinoString = ser.readline().decode("utf-8")
    dataArray = (arduinoString.split(','))

    accX.append(float(dataArray[0])/(32767/2))
    accX.pop(0)
    accY.append(float(dataArray[1])/(32767/2))
    accY.pop(0)
    accZ.append(float(dataArray[2])/(32767/2))
    accZ.pop(0)

    graphX.set_data(time, accX)
    graphY.set_data(time, accY)
    graphZ.set_data(time, accZ)

    return graphX, graphY, graphZ

delay = 20
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               interval=delay, blit=True)

plt.show()     

In [ ]:
ser.close()

<img src = "./image_files/MPU_subplot.png" style="border:1px solid black" width = 600>

# 4. Summary

In [25]:
%%html
<iframe src="https://www.youtube.com/embed/jemrZxfKVIY" 
width="560" height="315" frameborder="0" allowfullscreen></iframe>

In [26]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>